# installing the dependencies

In [1]:
!pip install ultralytics opencv-python numpy scipy tqdm matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

# unzip file with all the contents needed


In [3]:
!unzip '/content/Anushka-20250628T180458Z-1-001.zip'

Archive:  /content/Anushka-20250628T180458Z-1-001.zip
  inflating: Anushka/player_ids.json  
  inflating: Anushka/botsort.yaml    
  inflating: Anushka/player_appearance_log_resnet.json  
  inflating: Anushka/player_ids_final.json  
  inflating: Anushka/player_appearance_log.json  
  inflating: Anushka/detection_output.mp4  
  inflating: Anushka/tracking_output.mp4  
  inflating: Anushka/15sec_input_720p.mp4  
  inflating: Anushka/tacticam.mp4    
  inflating: Anushka/broadcast.mp4   
  inflating: Anushka/final_output_reid.mp4  
  inflating: Anushka/tracking_output_resnet.mp4  
  inflating: Anushka/best.pt         


# create yaml file for running inference


In [19]:
import yaml

tracker_config = {
    'tracker_type': 'botsort',
    'with_reid': True,
    'proximity_thresh': 0.35,
    'appearance_thresh': 0.5,
    'track_high_thresh': 0.7,
    'track_low_thresh': 0.05,
    'new_track_thresh': 0.75,
    'track_buffer': 400,
    'match_thresh': 0.9,
    'fuse_score': True,
    'gmc_method': 'sparseOptFlow',
    'model': 'auto'
}

with open('/content/Anushka/botsort.yaml', 'w') as f:
    yaml.dump(tracker_config, f, default_flow_style=False)

# inferencing code(player_reid.py)

In [20]:
from ultralytics import YOLO
import cv2
import os

# Load the YOLOv11 model
model = YOLO('/content/Anushka/best.pt')

# Define paths
video_path = '/content/Anushka/15sec_input_720p.mp4'
output_path = '/content/output_tracked_video.mp4'

# Verify video file exists
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file {video_path} not found")

# Initialize video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("Error opening video file")

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Check model class names to identify player class
print("Model class names:", model.names)

# Track players with BoT-SORT, ReID enabled, and confidence threshold
for result in model.track(
    source=video_path,
    conf=0.7,
    classes=[2],
    tracker='/content/Anushka/botsort.yaml',
    stream=True
):
    # Get the frame with bounding boxes and IDs
    frame = result.plot()

    # Write frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()


print(f"Output video saved as {output_path}")

Model class names: {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}

video 1/1 (frame 1/375) /content/Anushka/15sec_input_720p.mp4: 384x640 16 players, 54.0ms
video 1/1 (frame 2/375) /content/Anushka/15sec_input_720p.mp4: 384x640 15 players, 52.9ms
video 1/1 (frame 3/375) /content/Anushka/15sec_input_720p.mp4: 384x640 15 players, 42.5ms
video 1/1 (frame 4/375) /content/Anushka/15sec_input_720p.mp4: 384x640 14 players, 38.7ms
video 1/1 (frame 5/375) /content/Anushka/15sec_input_720p.mp4: 384x640 14 players, 39.0ms
video 1/1 (frame 6/375) /content/Anushka/15sec_input_720p.mp4: 384x640 14 players, 33.0ms
video 1/1 (frame 7/375) /content/Anushka/15sec_input_720p.mp4: 384x640 15 players, 32.9ms
video 1/1 (frame 8/375) /content/Anushka/15sec_input_720p.mp4: 384x640 15 players, 33.0ms
video 1/1 (frame 9/375) /content/Anushka/15sec_input_720p.mp4: 384x640 15 players, 32.7ms
video 1/1 (frame 10/375) /content/Anushka/15sec_input_720p.mp4: 384x640 15 players, 32.9ms
video 1/1 (frame 11/375